In [1]:
import pprint
import random
import pandas as pd
from faker import Faker
from faker.providers import profile
from datetime import datetime
from datetime import date

fake = Faker('es_MX')
fake.add_provider(profile)

CURRENCY_SIGN = '$'
# salario anual en dolares
MIN_SALARY = 15000.00
MAX_SALARY = 150000.00
# peso en kilogramos
MIN_WEIGHT = 50
MAX_WEIGHT = 120
# talla en centimetros
MIN_HEIGHT = 150
MAX_HEIGHT = 210


def generate_random_data(count):
    result_set = []
    for _ in range(count):
        user_info = fake.profile()
        user_info['country'] = fake.country()
        user_info['weight'] = int(random.uniform(MIN_WEIGHT, MAX_WEIGHT))
        user_info['height'] = int(random.uniform(MIN_HEIGHT, MAX_HEIGHT))
        user_info['salary'] = random.uniform(MIN_SALARY, MAX_SALARY)
        user_info['formatted_salary'] = CURRENCY_SIGN + str("{:,}".format(int(user_info['salary'])))

        result_set.append(user_info)
        #pprint.pprint(user_info)
    print(f'Generated {len(result_set)} records!')
    return result_set


def calculate_age(born):
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return int(today.year - born.year - ((today.month, today.day) < (born.month, born.day)))


#1. Generacion de datos aleatorios
data = generate_random_data(1000)

Generated 1000 records!


In [2]:
#2. Guardar los datos en un archivo separado por comas
raw_df = pd.DataFrame(data)
raw_df.to_csv('list.csv', index=False)

In [3]:
#3. Leer el archivo y crear dataFrame a partir de los datos contenidos
my_df = pd.read_csv('list.csv')

In [4]:
#4. Limpieza inicial de los datos
columns_to_delete = ['current_location', 'ssn', 'username', 'website', 'residence']
my_df.drop(columns=columns_to_delete, inplace=True)

In [5]:
# calculate dinamically some columns
my_df['age'] = my_df['birthdate'].apply(calculate_age)

In [6]:
# obtener dominios limpiando las direcciones de email

def extract_domain_from_email_address(email):
    import re
    domain = re.search("@[\w.]+", email)
    return domain.group()[1:]

my_df['domain'] = my_df['mail'].apply(extract_domain_from_email_address)


In [7]:
# quitar a cualquier persona que sea menor de edad y mayores de 100 años

my_df = my_df[(my_df['age'] >= 18) & (my_df['age'] < 100)]
my_df.reset_index(drop=True, inplace=True)

In [8]:
# #X. Exportar a formato HTML
print("Exportar DataFrame completo como HTML")
my_df.to_html('df_data.html', index=True)

Exportar DataFrame completo como HTML


In [9]:
# ordenar columnas alfabeticamente y las filas por edad (menor a mayor)
my_df.sort_values('age', ascending=True, inplace=True)
my_df.reset_index(drop=True, inplace=True)
my_df.sort_index(axis=1, inplace=True)

In [10]:
print('*** Reporte de cantidad personas por país de residencia ***')
df_by_country = my_df.groupby('country',as_index=True).agg({'country': ['count']})
print(df_by_country.head(100))
df_by_country.to_html('cantidad_personas_por_pais_residencia.html', index=True)

*** Reporte de cantidad personas por país de residencia ***
                     country
                       count
country                     
Afganistán                 2
Albania                    4
Alemania                   6
Andorra                    2
Angola                     6
Antigua y Barbuda          4
Arabia Saudita             4
Argelia                    2
Argentina                  2
Australia                  3
Austria                    5
Azerbaiyán                 3
Bahamas                    5
Bahrein                    3
Bangladesh                 3
Belarús                    1
Belice                     3
Benin                      3
Bhután                     5
Bolivia                    2
Bosnia y Herzegovina       3
Botswana                   4
Brasil                     7
Brunei Darussalam          2
Bulgaria                   4
Burkina Faso               7
Burundi                    6
Cabo Verde                 9
Camboya                    3
Camerún     

In [11]:
print('*** Reporte de cantidad personas por proveedor de email ***')
df_by_email_provider = my_df.groupby('domain', as_index=True).agg({'domain': ['count']})
print(df_by_email_provider.head(100))
df_by_email_provider.to_html('cantidad_personas_por_proveedor_email.html', index=True)

*** Reporte de cantidad personas por proveedor de email ***
            domain
             count
domain            
gmail.com      221
hotmail.com    247
yahoo.com      249


In [12]:
print('*** Reporte de cantidad personas por proveedor tipo de sangre ***')
df_by_blood_group = my_df.groupby('blood_group', as_index=True).agg({'blood_group': ['count']})
print(df_by_blood_group.head(100))
df_by_blood_group.to_html('cantidad_personas_por_grupo_sanguineo.html', index=True)

*** Reporte de cantidad personas por proveedor tipo de sangre ***
            blood_group
                  count
blood_group            
A+                   74
A-                   98
AB+                  84
AB-                  93
B+                  102
B-                   83
O+                   83
O-                  100


In [13]:
df_by_salary = my_df
print('Salario promedio anual : {:8.2f} dollars'.format(df_by_salary['salary'].mean()))

print('*** Reporte de promedios por pais [edad, peso, salario y talla] ***')
df_by_salary_grouped = df_by_salary.groupby(['country'], as_index=True).mean().add_suffix('_avg').reset_index()
df_by_salary_grouped.to_html('promedios_generales_por_pais.html', index=True)
print(df_by_salary_grouped)

group = df_by_salary.groupby('country')
#print(group.mean().head(100))

df_by_salary['salary'].hist(figsize=(12, 6), bins=25, grid=True);

Salario promedio anual : 83275.22 dollars
*** Reporte de promedios por pais [edad, peso, salario y talla] ***
                                 country    age_avg  height_avg  \
0                             Afganistán  74.500000  161.500000   
1                                Albania  43.250000  175.750000   
2                               Alemania  47.666667  175.166667   
3                                Andorra  63.000000  187.000000   
4                                 Angola  59.833333  186.333333   
5                      Antigua y Barbuda  48.750000  167.000000   
6                         Arabia Saudita  74.000000  189.500000   
7                                Argelia  54.000000  188.000000   
8                              Argentina  73.500000  171.000000   
9                              Australia  54.000000  189.000000   
10                               Austria  36.200000  179.000000   
11                            Azerbaiyán  50.666667  206.666667   
12                 